# Data Pre-processing

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

### Load data

In [2]:
# Read in data
df = pd.read_csv('results.txt', sep=",", header=None)
df.columns = ["id", "idx", "filename", "label", "time"]
df.head()

,id,idx,filename,label,time
0,23,1,1_0_2_20161219155701277.jpg.chip.jpg,Not smiling,0.8214
1,23,2,1_0_2_20161219153055452.jpg.chip.jpg,Not smiling,0.6173
2,23,3,1_1_2_20161219201453804.jpg.chip.jpg,Smiling,0.6046
3,23,4,1_0_2_20161219203122004.jpg.chip.jpg,Not smiling,0.6535
4,23,5,1_0_2_20161219201442357.jpg.chip.jpg,Not smiling,0.5369


### Filter data

In [3]:
# Create mask to filter away low and high reaction times
time_mask = (df['time'] > 0.2) & (df['time'] < 2)

# Filter out data
df = df[time_mask]

### Get normalized reaction times

In [4]:
# Create a new column with normalized times
scaler = MinMaxScaler()
for id_ in df['id'].unique():
    mask_ = df['id'] == id_
    scaler.fit(df.loc[mask_, ['time']])
    df.loc[mask_, 'time_norm'] = scaler.transform(df.loc[mask_, ['time']])
df.head()

,id,idx,filename,label,time,time_norm
0,23,1,1_0_2_20161219155701277.jpg.chip.jpg,Not smiling,0.8214,0.288367
1,23,2,1_0_2_20161219153055452.jpg.chip.jpg,Not smiling,0.6173,0.123597
2,23,3,1_1_2_20161219201453804.jpg.chip.jpg,Smiling,0.6046,0.113345
3,23,4,1_0_2_20161219203122004.jpg.chip.jpg,Not smiling,0.6535,0.152822
4,23,5,1_0_2_20161219201442357.jpg.chip.jpg,Not smiling,0.5369,0.058691


### Create the continuous index

In [5]:
# Define the scale (-scaling to scaling)
scaling = 300

# Create a mask for smile labels
smile_mask = df['label'] == 'Smiling'

# Create continuous index
df.loc[~smile_mask, 'cidx'] = (1 - df.loc[~smile_mask, 'time_norm']) * scaling
df.loc[smile_mask, 'cidx'] = (df.loc[smile_mask, 'time_norm'] - 1) * scaling
df.head()

,id,idx,filename,label,time,time_norm,cidx
0,23,1,1_0_2_20161219155701277.jpg.chip.jpg,Not smiling,0.8214,0.288367,213.489949
1,23,2,1_0_2_20161219153055452.jpg.chip.jpg,Not smiling,0.6173,0.123597,262.920804
2,23,3,1_1_2_20161219201453804.jpg.chip.jpg,Smiling,0.6046,0.113345,-265.996609
3,23,4,1_0_2_20161219203122004.jpg.chip.jpg,Not smiling,0.6535,0.152822,254.153548
4,23,5,1_0_2_20161219201442357.jpg.chip.jpg,Not smiling,0.5369,0.058691,282.392831
